# LSTM Model for Stock Price Prediction

## This NoteBook loads preprocessed data, defines an LSTM model using PyTorch, trains it, and evaluates its performance on stock price prediction.

## 1. Load Preprocessed Data

### Define Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
import joblib

#### Load the preprocessed sequences and scaler from the preprocessing script.

In [ ]:


X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')
scaler = joblib.load('scaler.pkl')
features_to_scale = joblib.load('../data/features_to_scale.pkl')

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

### Define the LSTM model architecture and set up the device (CPU/GPU).

#### Check for GPU availabiliy

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

#### Define LSTM Model

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = self.fc(out)
        return out

## 3. Prepare the Data for Training


### Convert data to PyTorch tensors and create data loaders.

### Convert NumPy arrays to PyTorch Tensors

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)


### Create TensorDataset and DataLoader

In [ ]:
batch_size = 64
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### Define Model Parameterss

In [ ]:
input_size = X_train.shape[2]
hidden_size = 128
num_layers = 2
output_size = 1
learning_rate = 0.001
num_epochs = 100

## 4. Train the Model

### Instantiate the model, define the loss function and optimizer, and train the model.

In [ ]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f"\nModel Architecture:\n{model}")
print(f"\nNumber of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

print("\n--- Starting Model Training ---")
train_losses = []
for epoch in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_losses.append(loss.item())
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.6f}')

print("--- Training Complete ---")

### Plot training loss

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True)
plt.show()


## 5. Evaluate and Visualize Predictions

### Evaluate the model on the test set and plot actual vs. predicted prices.

In [ ]:
print("\n--- Evaluating Model on Test Set ---")
model.eval()
test_predictions = []
actual_prices = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        test_predictions.extend(outputs.cpu().numpy().flatten())
        actual_prices.extend(batch_y.cpu().numpy().flatten())

### Calculate test loss

In [ ]:
test_loss = criterion(torch.tensor(test_predictions).to(device), torch.tensor(actual_prices).to(device))
print(f"Test Loss (MSE): {test_loss.item():.6f}")

### Conevert to NumPy arrays

In [ ]:
test_loss = criterion(torch.tensor(test_predictions).to(device), torch.tensor(actual_prices).to(device))
print(f"Test Loss (MSE): {test_loss.item():.6f}")

### Inverse transform Predictions

In [ ]:
close_col_idx = features_to_scale.index('close')
num_features = len(features_to_scale)
dummy_predictions_array = np.zeros((len(test_predictions_np), num_features))
dummy_predictions_array[:, close_col_idx] = test_predictions_np
original_predictions = scaler.inverse_transform(dummy_predictions_array)[:, close_col_idx]

### Inverse transform actual prices

In [ ]:
dummy_actual_array = np.zeros((len(actual_prices_np), num_features))
dummy_actual_array[:, close_col_idx] = actual_prices_np
original_actual_prices = scaler.inverse_transform(dummy_actual_array)[:, close_col_idx]

### Plot Actual vs. Predicted Prices

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(original_actual_prices, label='Actual Prices (Original Scale)', color='blue')
plt.plot(original_predictions, label='Predicted Prices (Original Scale)', color='red', linestyle='--')
plt.title('Actual vs. Predicted Stock Prices')
plt.xlabel('Time Step (Test Set)')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


### Plot a smaller segment

In [ ]:
plt.figure(figsize=(16, 8))
plot_start_idx = 0
plot_end_idx = 100
plt.plot(original_actual_prices[plot_start_idx:plot_end_idx], label='Actual Prices (Original Scale)', color='blue')
plt.plot(original_predictions[plot_start_idx:plot_end_idx], label='Predicted Prices (Original Scale)', color='red', linestyle='--')
plt.title(f'Actual vs. Predicted Stock Prices (First {plot_end_idx} Test Samples)')
plt.xlabel('Time Step (Test Set)')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
print("\n--- PyTorch LSTM Training and Evaluation Complete ---")